In [1]:
import pandas as pd
import pickle
from utils import data_preprocess 

with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\df_final.pkl', 'rb') as f:
    df = pickle.load(f)
    
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\df_final_raw.pkl', 'rb') as f:
    df_raw = pickle.load(f)
    

In [2]:
df.shape, df_raw.shape

((524117, 30), (524117, 3))

In [3]:
print(df.shape)
df.head()

(524117, 30)


,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,...,log_99_ema,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 20:15:00,0.401930,0.001804,2.406179,0.000063,-0.000455,-0.000250,-0.000080,0.000019,0.000373,-0.000146,...,-0.000228,-0.000111,-0.000055,-41.318663,-47.808444,-7.849202e-06,0.444260,-0.111538,-4.903050,6.489781
2022-11-14 20:16:00,-0.322941,0.000225,-0.875063,0.000342,-0.000482,-0.000243,-0.000081,0.000020,0.000336,-0.000118,...,-0.000218,-0.000109,-0.000054,-37.196986,-45.686152,-6.308475e-06,0.447616,-0.048051,-1.803474,8.489166
2022-11-14 20:17:00,0.047696,0.001045,3.640008,0.000262,-0.000404,-0.000231,-0.000075,0.000020,0.000513,-0.000028,...,-0.000190,-0.000101,-0.000052,-32.190122,-42.986946,-4.694262e-06,0.427829,-0.003950,-1.147692,10.796824
2022-11-14 20:18:00,0.751147,0.000294,-0.718221,0.000422,-0.000253,-0.000242,-0.000069,0.000020,0.000459,-0.000003,...,-0.000180,-0.000098,-0.000051,-27.518817,-39.893321,-1.867537e-06,0.437425,0.011163,14.748759,12.374503
2022-11-14 20:19:00,-0.154670,0.000703,1.388568,0.000675,-0.000139,-0.000222,-0.000065,0.000020,0.000520,0.000051,...,-0.000160,-0.000093,-0.000050,-22.633165,-36.441289,5.219766e-07,0.524746,0.003284,-1.205912,13.808125


In [4]:
print(df_raw.shape)
df_raw.head()

(524117, 3)


,volume,mid_price,pct_change
time,,,
2022-11-14 20:15:00,56.890409,16233.560,0.001804
2022-11-14 20:16:00,38.518180,16237.215,0.000225
2022-11-14 20:17:00,40.355329,16254.185,0.001045
2022-11-14 20:18:00,70.668109,16258.970,0.000294
2022-11-14 20:19:00,59.737850,16270.405,0.000703


In [5]:
df_raw.iloc[0].name

Timestamp('2022-11-14 20:15:00')

In [11]:
df.isna().sum()

volume           0
mid_price        0
log_returns      0
7min_MA          0
25min_MA         0
99min_MA         0
300min_MA        0
900min_MA        0
7_ema            0
25_ema           0
99_ema           0
300_ema          0
900_ema          0
log_7min_MA      0
log_25min_MA     0
log_99min_MA     0
log_300min_MA    0
log_900min_MA    0
log_7_ema        0
log_25_ema       0
log_99_ema       0
log_300_ema      0
log_900_ema      0
MACD             0
Signal_Line      0
obv              0
rsi              0
vma              0
vol_pct          0
MACD-SIGNAL      0
dtype: int64

In [6]:
df_final, df_raw = data_preprocess(df, df_raw, sequence_length=64)

  0%|          | 0/524054 [00:00<?, ?it/s]

100%|██████████| 524054/524054 [00:02<00:00, 200074.60it/s]

(524054, 30, 64) (524054, 3)


In [34]:
from collections import deque
from datetime import datetime


class Environment:
    def __init__(self, chart_data=None, raw_data=None,chart_index:list = None, risk_adverse= 1.3 ,balance=100000000, transaction=0.0004, max_leverage=10):
        self.chart_data = chart_data # numpy array
        self.raw_data = raw_data
        self.chart_index = chart_index # python list
        self.idx = 0

        self.risk_adverse = risk_adverse # 손실에 주는 가중치 
        self.transaction = transaction # 거래수수료
        self.max_leverage = max_leverage
        
        self.current_state = self.chart_data[self.idx]
        self.next_state = self.chart_data[self.idx+1]
                    
        self.current_price = self.raw_data["mid_price"].iloc[self.idx]
        self.next_price = self.raw_data["mid_price"].iloc[self.idx+1]
        
        self.initial_balance = balance
        self.balance = [balance]  # 포트폴리오가 보유한 현금
        self.bitcoin = [0]  # 포트폴리오가 보유한 비트코인의 가치 (매 거래마다 바로 청산됨)
        self.portfolio_value = []
        
        self.action_info = [-3, -1, 0, 1, 3]
        self.action_size = len(self.action_info)
        self.action_list = deque([len(self.action_info)//2 for i in range(32)]) # 이전 32분을 저장함.

        self.position = 0 # 이전의 position 비율을 저장하는 변수. (+)는 long, (-)는 short
        self.position_list = deque([0 for i in range(32)]) # 이전 32분을 저장함.
        
        # self.profit_queue = deque([0.0004 for i in range(64)]) # 이전 1시간의 변동성을 고려함
        
        
    def reset(self):
        self.idx = 0
        state = self.chart_data[self.idx]
        return state
    
    def get_profit_std(self, profit):
        self.profit_queue.popleft()
        self.profit_queue.append(profit)
        std = (pow(sum(self.profit_queue),2)/len(self.profit_queue))**0.5
        return abs(std)
    
    
    def step(self, action):
        self.current_state = self.chart_data[self.idx]
        self.next_state = self.chart_data[self.idx+1]                    
        self.current_price = self.raw_data["mid_price"].iloc[self.idx]
        self.next_price = self.raw_data["mid_price"].iloc[self.idx+1]

        current_value = self.balance[-1] + self.bitcoin[-1]*self.current_price 
        self.portfolio_value.append(current_value)
        
        s_prime = self.chart_data[self.idx+1]
        
        # action list에 새로운 action 추가해줌
        self.action_list.append(action)
        self.action_list.popleft()
            
        # reward 계산
        profit = self.get_reward(action)
        
        # 얻은 수익률의 표준편차를 구해준다.
        # std = self.get_profit_std(profit)  
            
        # sharpe ratio를 maximize하는 형식
        reward = profit/0.001
        
        # risk adverse정도를 고려해서 reward 계산
        if reward<0:
            reward =  reward * self.risk_adverse
        
        current_time = self.raw_data.iloc[self.idx].name
        current_day = datetime.strftime(current_time, '%Y-%m-%d')
        next_time = self.raw_data.iloc[self.idx+1].name
        next_day = datetime.strftime(next_time, '%Y-%m-%d')

        # 시간 index 갱신
        self.idx += 1

        if current_day == next_day:
            return {"state_time":current_day, 
                    "next_state":s_prime, 
                    "reward":round(reward,8), 
                    "done":False, 
                    "portfolio_value":self.portfolio_value[-1], 
                    "balance":self.balance[-1], 
                    "bitcoin":self.bitcoin[-1], 
                    "position_list":self.position_list,
                    "action_list":self.action_list,
                    "current_price": self.current_price,
                    "index": self.idx}
        else:
            print(f'{current_day}에서 {format(round(self.portfolio_value[-1]), ",")}원으로 trading stop, {round((self.portfolio_value[-1]/100000000-1)*100,1)}%')
            print("#########################################################################")
            
            P = self.portfolio_value[-1]
            B = self.balance[-1]
            C = self.bitcoin[-1]
            
            # 다음날로 넘어갈 때 포지션 청산 
            self.position = 0
            
            # 다음날 거래는 다시 1억으로 시작
            self.balance = [self.balance[0]]  # 포트폴리오가 보유한 현금
            self.bitcoin = [0]  # 포트폴리오가 보유한 비트코인의 가치 (매 거래마다 바로 청산됨)
            self.portfolio_value = []
            
            return {"state_time":current_day, 
                    "next_state":s_prime, 
                    "reward":round(reward,8), 
                    "done":True, 
                    "portfolio_value":P, 
                    "balance":B, 
                    "bitcoin":C, 
                    "position":self.position,
                    "action_list":self.action_list,
                    "current_price": self.current_price,
                    "index": self.idx}
    
    
    
    def position_calc(self, action): # max_leverage를 고려해서 position을 계산해주는 함수
        action = self.action_info[action] # action의 실제 action
        execute_position = self.position
        
        if action * self.position > 0:
            if  self.position + action > self.max_leverage:
                ratio = 0
                self.position = self.max_leverage
                return self.position, ratio, False
            
            elif self.position + action < -self.max_leverage:
                ratio  = 0 
                self.position = -self.max_leverage
                return self.position, ratio, False
            else:
                self.position += action
                ratio = action
                return self.position, ratio, False
                    
        elif action * self.position < 0:
            self.position = action 
            ratio = action 
            return self.position, ratio, True
        
        elif action*self.position==0:
            if action==0:
                self.position = action 
                return self.position, 0, False
            
            elif self.position==0:
                self.position = action 
                return self.position, self.position, False
            
        
    def get_reward(self, action):     
        pre_position = self.position
        
        
        # Short
        if action < (self.action_size//2):
            position, ratio, execution = self.position_calc(action)    
            
            self.position_list.append(position)
            self.position_list.popleft()

            sell_budget = self.initial_balance * ratio
            if execution: 
                # 직전 포지션이 long, clearing_budget: (+)
                clearing_budget = (self.bitcoin[-1]) * self.current_price
                # 이전에 매수를 한 경우 => 현재 매도(long 청산)
                self.balance.append(self.balance[-1] - sell_budget*(1-self.transaction) + clearing_budget*(1-self.transaction)) # clearing budget logic이 오류가 있음.
                self.bitcoin.append(sell_budget/self.current_price)          
            else: # 직전 포지션이 short
                self.balance.append(self.balance[-1] - sell_budget*(1-self.transaction))
                self.bitcoin.append(self.bitcoin[-1] + sell_budget/self.current_price)  
        
            current_value = self.portfolio_value[-1]
            next_value = self.balance[-1] + self.bitcoin[-1]*(self.next_price)
            reward = (next_value - current_value) / self.initial_balance
            reward = 1 if reward >= 0 else -1
            return reward
            
        # Long
        elif action > (self.action_size//2):
            position, ratio, execution = self.position_calc(action)
            
            self.position_list.append(position)
            self.position_list.popleft()

            buy_budget = self.initial_balance * ratio
    
            if execution: # 직전 포지션 short | clearing_budget: (-) 
                clearing_budget = (self.bitcoin[-1]) * self.current_price
                self.balance.append(self.balance[-1] - buy_budget*(1+self.transaction) + clearing_budget*(1+self.transaction)) 
                self.bitcoin.append(buy_budget/self.current_price)              

            else: # 직전 포지션 long
                self.balance.append(self.balance[-1] - buy_budget*(1+self.transaction))
                self.bitcoin.append(self.bitcoin[-1] + buy_budget/self.current_price)  
            
            current_value = self.portfolio_value[-1]
            next_value = self.balance[-1] + self.bitcoin[-1]*(self.next_price)
            reward = (next_value - current_value) / self.initial_balance
            reward = 1 if reward >= 0 else -1
            return reward
        
        # 청산
        elif action == (self.action_size//2):
            self.position_list.append(0)
            self.position_list.popleft()
            
            if pre_position>=0:
                clearing_budget = (self.bitcoin[-1]) * self.current_price
                self.balance.append(self.balance[-1] + clearing_budget*(1-self.transaction))
                self.bitcoin.append(0)
                
            elif pre_position<0: 
                clearing_budget = (self.bitcoin[-1]) * self.current_price
                self.balance.append(self.balance[-1] + clearing_budget*(1+self.transaction))
                self.bitcoin.append(0)
            
            return 0


In [35]:
env = Environment4(df_final, df_raw, risk_adverse = 1.2, transaction=0.0004)
reset = env.reset()

In [40]:
env.step(0)

{'state_time': '2022-11-14',
 'next_state': array([[-3.42243092e-01,  3.87552072e-01, -3.92649313e-01, ...,
          4.13818483e-02, -4.04939277e-01,  4.07499575e-01],
        [-2.19724094e-04, -3.59627537e-04, -4.64301660e-05, ...,
         -4.30377878e-04,  2.44547630e-04,  3.06375740e-06],
        [-1.31256111e+00,  6.36837787e-01, -8.70913960e-01, ...,
          9.03808971e+00, -1.56802434e+00, -9.87470223e-01],
        ...,
        [-2.78716780e-02, -3.52479821e-03, -1.30350747e-02, ...,
         -1.00956281e-02, -3.04983075e-02,  8.86527358e-04],
        [ 1.21272635e+00, -2.13238830e+00, -2.01315240e+00, ...,
         -1.13696833e+00, -1.07854324e+01, -2.00632267e+00],
        [ 1.40869264e+01,  1.34500019e+01,  1.25670023e+01, ...,
          4.15212745e+00,  3.35997810e+00,  2.57229995e+00]]),
 'reward': -0.03308165,
 'done': False,
 'portfolio_value': 99828565.10578954,
 'balance': 999640000.0,
 'bitcoin': -55136.078923261484,
 'position': -10,
 'action_list': deque([2,
     